In [0]:
# from google.colab import files
# uploaded = files.upload()
import pandas as pd

# prepare cells to show 1000 characters
pd.set_option("display.max_colwidth",1000)

rdt = pd.read_csv("https://raw.githubusercontent.com/haoshuai999/Conversation-Bot/master/trump_tweets_year.csv")

In [0]:
from re import match, IGNORECASE, search, escape

In [0]:
dirty_words = pd.read_fwf("https://raw.githubusercontent.com/haoshuai999/Conversation-Bot/master/en.txt", header=None)
dirty_words = dirty_words[0].tolist()
dirty_words

In [0]:
from random import choice
 
reflections = {
    "am": "are",
    "was": "were",
    "i": "you",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will",
    "my": "your",
    "are": "am",
    "you've": "I have",
    "you'll": "I will",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "me": "you"
}
 
actions = [
    [r'(.*) Times|(.*) Tribune|(.*) Post|(.*) Journal',
     [(index,s) for index, s in rdt['text'].iteritems() if search(r"fake news",s,IGNORECASE)]],
    
    [r'(.*) Mueller Report(.*)',
     [(index,s) for index, s in rdt['text'].iteritems() if search(r"witch hunt",s,IGNORECASE)]],
    
    [r'(.*) Hillary(.*)',
     [(index,s) for index, s in rdt['text'].iteritems() if search(r"hillary",s,IGNORECASE)]],
    
    [r'(.*) wall(.*)|(.*) immigrant(.*)|(.*) immigration(.*)',
     [(index,s) for index, s in rdt['text'].iteritems() if search(r"\bwall\b|\bimmigrant\b|\bimmigration\b",s,IGNORECASE)]],
    
    [r'(.*) impeachment(.*)|(.*) impeach(.*)',
    ["10-01-2019 16:41:00\nAs I learn more and more each day, I am coming to the conclusion that what is taking place is not an impeachment, it is a COUP, intended to take away the Power of the....",
    "09-29-2019 21:11:00\nNancy Pelosi and the Democrats can’t put down the Impeachment match. They know they couldn’t beat him in 2016 against Hillary Clinton, and they’re increasingly aware of the fact that they won’t win against him in 2020, and Impeachment is the only tool they have to get....",
    "09-29-2019 20:22:00\nAll that’s swirling around us now is Impeachment. We talk about it day and night, it’s what’s on the news, there is NOTHING that has turned up that is Impeachable. Our founding fathers set impeachment to be extremely rare. We need to get good stuff done. Let the people vote,...."
    ]],
    
    [r'Hello(.*)',
     ["Hello. What can I help you?",
      "Hi there."]],
    
    [r'\bYes\b|\bNo\b',
     ["You are supposed to ask me questions.",
      "I am not asking you. I am displaying the tweets."]],
    
    [r'quit',
     ["Thank you for talking with me.",
      "Good-bye.",
      "More tweet can found on https://twitter.com/realDonaldTrump"]]
]
 
 
# def reflect(fragment):
    
#     # Turn a string into a series of words
#     tokens = fragment.lower().split()
    
#     # for each word...
#     for i in range(len(tokens)):
#         token = tokens[i]
    
#         # see if the word is in the "reflections" list and if it
#         # is, replace it with its reflection (you -> me, say)
#         if token in reflections:
#             tokens[i] = reflections[token]
            
#     return ' '.join(tokens)
 
 
def respond(statement):
    if len(statement) > 100:
        return "Could you summerize your reply? It is too long for me to understand."
    
    # run through all the actions
    for j in range(len(actions)):

        # for each one, see if it matches the statment that was typed
        pattern = actions[j][0] 
        responses = actions[j][1]
        found = match(pattern, statement.rstrip(".!"),IGNORECASE)

        if found:
            # for the first match, select a response at random and insert
            response = choice(responses)

            if isinstance(response, tuple):
                print(rdt['created_at'][response[0]])
                return response[1].format(*[g for g in found.groups()])
            else:
                return response.format(*[g for g in found.groups()])
    
    return puresearch(statement)

def puresearch(statement):
    regex = r"\b(?=\w)" + escape(statement) + r"\b(?!\w)"
    responses = [(index,s) for index, s in rdt['text'].iteritems() if search(regex,s,IGNORECASE)]
    if responses != []:
      response = choice(responses)
      print(rdt['created_at'][response[0]])
      return response[1]
    else:
      return "I cannot find related results."
 
 
def trump():
    # a friendly welcome
    print("Hello. I am a Trump Tweet bot. I can answer any questions using President Trump's tweets. What do you want to ask?")
    print("If you just want to search tweet, add 'SEARCH' in the beginning of your input. Type 'quit' to leave.")
 
    # talk forever...
    while True:
        
        # collect a statement and respond, stop the conversation on 'quit'
        statement = input("> ")
        for word in dirty_words:
            regex_dirty = r"\b(?=\w)" + escape(word) + r"\b(?!\w)"
            if search(regex_dirty,statement,IGNORECASE):
                return "You are so rude. I don't want to talk to you."
                break
        if statement[:6] == 'SEARCH':
            print(puresearch(statement.replace("SEARCH","").strip()))
        else:
            print(respond(statement))
 
        if statement == "quit":
            break


In [0]:
trump()